In [7]:
import face_recognition
import cv2
import numpy as np
import pandas
import csv

In [8]:
#Using Pandas to read names and pictures in our database
#If you want to add picture to data base open read me
df = pandas.read_csv('employees.csv')

names = df['Name']

address = df['Address']

known_face_encodings = []

known_face_names = []

for img in address:
    image = cv2.imread(img , cv2.COLOR_RGB2GRAY)
    face_encoding = face_recognition.face_encodings(image)[0]
    known_face_encodings.append(face_encoding)
        
for title in names:
    known_face_names.append(title)    

In [9]:
#Capturing Video with web cam
video_capture = cv2.VideoCapture(0)

face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    gray_frame = cv2.cvtColor(frame , cv2.COLOR_BGR2GRAY)
    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(gray_frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame , cv2.COLOR_BGR2RGB)
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        
        
        
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        
        
        if name == 'Unknown':
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.rectangle(frame, (20 , 5) ,(550 , 50) , (0, 0, 255), cv2.FILLED)
            cv2.putText(frame, 'Your profile is not registered, you are not allowed to enter.', (30 , 30), font, 0.5, (255, 255, 255), 1)
        else:
        
            name = f'{name} welcome'
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name , (left + 6 ,bottom  -6), font, 0.5, (255, 255, 255), 1)
        
    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()